In [55]:
import re
from collections import OrderedDict
import numpy as np

class sentistrength:
    def __init__(self, config=dict()):
        self.negasi = [line.replace('\n','') for line in open("negatingword.txt").read().splitlines()]
        self.tanya = [line.replace('\n','') for line in open("questionword.txt").read().splitlines()]
        #create sentiment words dictionary
        self.sentiwords_txt = [line.replace('\n','').split(":") for line in open("sentiwords_id.txt").read().splitlines()]
        self.sentiwords_dict = OrderedDict()
        for term in self.sentiwords_txt:
            self.sentiwords_dict[term[0]] = int(term[1])
        #create emoticon dictionary
        self.emoticon_txt = [line.replace('\n','').split(" | ") for line in open("emoticon_id.txt").read().splitlines()]
        self.emoticon_dict = OrderedDict()
        for term in self.emoticon_txt:
            self.emoticon_dict[term[0]] = int(term[1])
        #create idioms dictionary
        self.idioms_txt = [line.replace('\n','').split(":") for line in open("idioms_id.txt").read().splitlines()]
        self.idioms_dict = OrderedDict()
        for term in self.idioms_txt:
            self.idioms_dict[term[0]] = int(term[1])
        #create boosterwords dictionary
        self.boosterwords_txt = [line.replace('\n','').split(":") for line in open("boosterwords_id.txt").read().splitlines()]
        self.boosterwords_dict = OrderedDict()
        for term in self.boosterwords_txt:
            self.boosterwords_dict[term[0]] = int(term[1])
        self.negation_conf = config["negation"]
        self.booster_conf = config["booster"]
        self.ungkapan_conf = config["ungkapan"]
        self.consecutive_conf = config["consecutive"]
        self.repeated_conf = config["repeated"]
        self.emoticon_conf = config["emoticon"]
        self.question_conf = config["question"]
        self.exclamation_conf = config["exclamation"]
        self.punctuation_conf = config["punctuation"]
        self.mean_conf = False

    def senti(self,term):
        try:
            return self.sentiwords_dict[term]
        except:
            return 0

    def emosikon(self,term):
        try:
            return self.emoticon_dict[term]
        except:
            return 0

    def ungkapan(self,term):
        try:
            return self.idioms_dict[term]
        except:
            return 0

    def booster(self, term):
        try:
            return self.boosterwords_dict[term]
        except:
            return 0

    def cek_negationword(self, prev_term, prev_term2):
        #jika kata sebelumnya (index-1) adalah kata negasi, negasikan nilai -+nya
        if prev_term in self.negasi or prev_term2+" "+prev_term in self.negasi:
            # print prev_term
            self.score = -abs(self.score) if self.score>0 else abs(self.score)

    def cek_boosterword(self,term):
        booster_score = self.booster(term)
        if booster_score !=0 and self.score>0: self.score += booster_score
        if booster_score !=0 and self.score<0: self.score -= booster_score

    def cek_consecutive_term(self, prev_term):
        if self.prev_score>0 and self.score >=3: self.score+=1 
        if self.prev_score<0 and self.score <=-3: self.score-=1 

    def cek_ungkapan(self, bigram,trigram, i):
        bigram = ' '.join(bigram)
        trigram = ' '.join(trigram)
        ungkapan_score = self.ungkapan(bigram)
        if ungkapan_score==0:
            ungkapan_score = self.ungkapan(trigram)
        if ungkapan_score!=0:
            self.score = ungkapan_score
            self.prev_score = 0
            self.pre_max_pos[i-1] = 1
            self.pre_max_neg[i-1] = -1
            self.max_pos = self.pre_max_pos[i-2] #if len(self.pre_max_pos)>1 else 1
            self.max_neg = self.pre_max_neg[i-2] #if len(self.pre_max_neg)>1 else -1
            self.sentence_score[i-1] = re.sub(r'\[\d\]','',self.sentence_score[i-1])

    def cek_repeated_punctuation(self, next_term):
        if re.search(r'!{2,}',next_term) and self.score >=3: self.score+=1
        if re.search(r'!{2,}',next_term) and self.score <=-3: self.score-=1

    def remove_extra_repeated_char(self, term):
        return re.sub(r'([A-Za-z])\1{2,}',r'\1',term)
    def plural_to_singular(self, term):
        return re.sub(r'([A-Za-z]+)\-\1', r'\1',term)
    def classify(self):
        result = "netral"
        try:
            if self.mean_conf:
                mean_p = np.mean(self.mean_pos)
                mean_n = np.mean(self.mean_neg)
                print(mean_p, mean_n)
                if mean_p > mean_n:
                    result = "positif"
                elif mean_p < mean_n and not self.is_tanya:
                    result = "negatif"
                elif mean_p < mean_n and self.is_tanya:
                    result = "netral"
            else:
                if abs(self.sentences_max_pos) > abs(self.sentences_max_neg):
                    result = "positif"
                elif abs(self.sentences_max_pos) < abs(self.sentences_max_neg):
                    result = "negatif"
                elif abs(self.sentences_max_pos) == abs(self.sentences_max_neg):
                    result = "netral"
        except:
            print("error ",self.sentences_max_pos, self.sentences_max_neg)
        return result
    def cek_neutral_term(self,terms,i):
        if terms[i-1] in self.neutral_term or terms[i+1] in self.neutral_term: self.score=1 

    def main(self,sentence):
        self.neutral_term = ['jika','kalau']
        sentences = sentence.split('.')
        self.sentences_max_neg = -1
        self.sentences_max_pos = 1
        self.sentences_score = []
        self.sentences_text = []
        for sentence in sentences:
            self.max_neg = -1
            self.max_pos = 1
            self.mean_neg = [1]
            self.mean_pos = [1]
            self.sentence_score=[]
            terms = sentence.split()
            # terms = re.split(r'[\s,.]',sentence)
            terms_length = len(terms)
            self.is_tanya = False
            self.sentence_text = ''
            # print self.max_pos, self.max_neg
            #SEMUA KALIMAT YANG MEMILIKI TANDA SERU MEMILIKI +ve minimal 2
            if self.exclamation_conf and re.search('!',sentence): self.max_pos = 2
            self.prev_score = 0
            self.pre_max_pos = []
            self.pre_max_neg = []
            for i,term in enumerate(terms):
                # repeated_term = ''
                is_extra_char = False
                plural = ''
                self.score = 0
                # if re.search(r'[A-Za-z\-.]+',term):
                # print term
                if re.search(r'([A-Za-z])\1{3,}',term):
                    is_extra_char = True
                    # repeated_term =term
                term = self.remove_extra_repeated_char(term)
                if re.search(r'([A-Za-z]+)\-\1',term):
                    plural = term
                    term = self.plural_to_singular(term)
                #GET SENTI SCORE#
                self.score = self.senti(term)
                # print "senti score",term, self.score

                #NEGATION HANDLER#
                if self.negation_conf and self.score !=0 and i>0:self.cek_negationword(terms[i-1],terms[i-2])
                # print  "negation score",term, self.score

                #BOOSTERWORD HANDLER#
                if self.booster_conf and self.score !=0 and i>0 and i<=(terms_length-1):self.cek_boosterword(terms[i-1])
                if self.booster_conf and self.score !=0 and i>=0 and i<(terms_length-1):self.cek_boosterword(terms[i+1])
                # print  "booster score",term, self.score

                #IDIOM/UNGKAPAN HANDLER#
                if self.ungkapan_conf and i>0 and i<=(terms_length-1):self.cek_ungkapan([terms[i-1],term],[terms[i-2],terms[i-1],term],i)
                # if self.ungkapan_conf and i>=0 and i<(terms_length-1):self.cek_ungkapan([term,terms[i+1]])
                # print  "idiom score",term, self.score

                #CONSECUTIVE SENTIMENT WORD#
                if self.consecutive_conf and i>0 and i<=(terms_length-1) and self.score !=0:self.cek_consecutive_term(terms[i-1])
                # print  "consecutive score",term, self.score

                #+1 SENTI SCORE IF REPEATED CHAR ON POSITIVE/NEGATIVE +2 IF NEUTRAL TERM
                if self.repeated_conf and is_extra_char==True and self.score>0: self.score+=1
                if self.repeated_conf and is_extra_char==True and self.score<0: self.score-=1
                if self.repeated_conf and is_extra_char==True and self.score==0: self.score=2
                # print  "repeat char score", term, self.score
                if self.punctuation_conf and i>=0 and i<(terms_length-1): self.cek_repeated_punctuation(terms[i+1])
                # CEK APAKAH TERDAPAT KATA TANYA
                if self.question_conf and (term in self.tanya or re.search(r'\?',term)):self.is_tanya = True
                # CEK neutral term 
                if self.score!=0 and i>1 and i<(terms_length-2): self.cek_neutral_term(terms,i)
                # if self.score!=0 and i>0 and i<(terms_length-4): self.cek_neutral_term(terms,i)
                if self.emoticon_conf and self.score==0: self.score = self.emosikon(term)

                self.prev_score = self.score
                if self.mean_conf and self.score>0: self.mean_pos.append(self.score)	
                if self.mean_conf and self.score<0: self.mean_neg.append(abs(self.score))
                #GET MAX SCORE +ve/-ve	
                self.max_pos= self.score if self.score > self.max_pos else self.max_pos
                self.max_neg= self.score if self.score < self.max_neg else self.max_neg
                #insert score info current term
                self.pre_max_pos.append(self.max_pos)
                self.pre_max_neg.append(self.max_neg)
                # print self.pre_max_pos, self.pre_max_neg
                if plural !='': term = plural
                self.sentence_text += ' {}'.format(term)
                if self.score != 0:term = "{} [{}]".format(term, self.score)
                self.sentence_score.append(term)

            self.sentences_text.append(self.sentence_text)
            self.sentences_score.append(" ".join(self.sentence_score))
            if self.is_tanya: 
                self.max_neg = -1
            self.sentences_max_pos = self.max_pos if self.max_pos > self.sentences_max_pos else self.sentences_max_pos
            self.sentences_max_neg = self.max_neg if self.max_neg < self.sentences_max_neg else self.sentences_max_neg
            # print self.sentences_max_pos, self.sentences_max_neg
        sentence_result = self.classify()
        # print self.sentences_text
        return {"classified_text":". ".join(self.sentences_score),"tweet_text":". ".join(self.sentences_text),"sentence_score":self.sentences_score,"max_positive":self.sentences_max_pos,"max_negative":self.sentences_max_neg,"kelas":sentence_result}

In [56]:
config = dict()
config["negation"] = True
config["booster"]  = True
config["ungkapan"]  = True
config["consecutive"]  = True
config["repeated"]  = True
config["emoticon"]  = True
config["question"]  = True
config["exclamation"]  = True
config["punctuation"]  = True

In [57]:
senti = sentistrength(config)

In [58]:
# Load data ke dalam data frame 
import pandas as pd

# spesifikasi encoding diperlukan karena data tidak menggunakan UTF-8
df = pd.read_csv('lemma_150.csv', encoding='latin-1')

df.head()

,Unnamed: 0,Review,Sentimen,Review_norm,review_tokens,review_tokens_SR,review_tokens_stemmed
0,0,it beautiful package bintang buat safety nya,Positif,itu beautiful package bintang buat safety nya,"['itu', 'beautiful', 'package', 'bintang', 'bu...","['beautiful', 'package', 'bintang', 'buat', 's...","['beautiful', 'package', 'bintang', 'buat', 's..."
1,1,tipe kulitku kering,Netral,tipe kulitku kering,"['tipe', 'kulitku', 'kering']","['tipe', 'kulitku', 'kering']","['tipe', 'kulit', 'kering']"
2,2,pengiriman agak lama karena beda provinsi tapi...,Negatif,pengiriman agak lama karena beda provinsi tapi...,"['pengiriman', 'agak', 'lama', 'karena', 'beda...","['pengiriman', 'lama', 'beda', 'provinsi', 'ga...","['kirim', 'lama', 'beda', 'provinsi', 'gapapa'..."
3,3,top,Positif,top,['top'],['top'],['top']
4,4,tipe kulitku berminyak tipe kulitku kering tip...,Positif,tipe kulitku berminyak tipe kulitku kering tip...,"['tipe', 'kulitku', 'berminyak', 'tipe', 'kuli...","['tipe', 'kulitku', 'berminyak', 'tipe', 'kuli...","['tipe', 'kulit', 'minyak', 'tipe', 'kulit', '..."


In [59]:
# Drop kolom yang tidak terpakai
df = df.drop(df.columns[[0, 1, 3, 4, 5]], axis=1)

# Cek data teratas
df.head()

,Sentimen,review_tokens_stemmed
0,Positif,"['beautiful', 'package', 'bintang', 'buat', 's..."
1,Netral,"['tipe', 'kulit', 'kering']"
2,Negatif,"['kirim', 'lama', 'beda', 'provinsi', 'gapapa'..."
3,Positif,['top']
4,Positif,"['tipe', 'kulit', 'minyak', 'tipe', 'kulit', '..."


In [60]:
# Mengubah data kolom menjadi kalimat utuh tanpa tanda koma, petik, dan tanda kurung siku
df['review_tokens_stemmed'] = df['review_tokens_stemmed'].apply(lambda x: ' '.join(eval(x)))

# Menampilkan hasil
df.head()

,Sentimen,review_tokens_stemmed
0,Positif,beautiful package bintang buat safety nya
1,Netral,tipe kulit kering
2,Negatif,kirim lama beda provinsi gapapa packingnya ama...
3,Positif,top
4,Positif,tipe kulit minyak tipe kulit kering tipe kulit...


In [61]:
# Memindahkan kolom sentimen setelah kolom review
columns = list(df.columns)
sentimen_index = columns.index('Sentimen')
review_index = columns.index('review_tokens_stemmed')
columns.insert(review_index + 1, columns.pop(sentimen_index))

# Menggunakan reindex untuk mengubah urutan kolom
df = df.reindex(columns=columns)

# Cek data teratas
df.head()


,review_tokens_stemmed,Sentimen
0,beautiful package bintang buat safety nya,Positif
1,tipe kulit kering,Netral
2,kirim lama beda provinsi gapapa packingnya ama...,Negatif
3,top,Positif
4,tipe kulit minyak tipe kulit kering tipe kulit...,Positif


In [62]:
# import hasil menjadi csv
df.to_csv("data150_new.csv", index=False)

In [63]:
import pandas as pd

# Membaca file CSV
df = pd.read_csv('data150_new.csv')

# Nama kolom yang ingin diubah dan dianalisis sentimennya
nama_kolom = 'review_tokens_stemmed'

# Fungsi untuk mendapatkan sentimen dari setiap baris dalam kolom
def analisis_sentimen(sentence):
    result = senti.main(sentence)
    return result

# Menggunakan fungsi apply untuk menerapkan analisis sentimen pada setiap baris
df['sentimen'] = df[nama_kolom].apply(analisis_sentimen)

# Menyimpan perubahan ke dalam file CSV
df.to_csv('sentimen_150.csv', index=False)

# Menampilkan DataFrame setelah perubahan
print(df)


                                 review_tokens_stemmed Sentimen  \
0            beautiful package bintang buat safety nya  Positif   
1                                    tipe kulit kering   Netral   
2    kirim lama beda provinsi gapapa packingnya ama...  Negatif   
3                                                  top  Positif   
4    tipe kulit minyak tipe kulit kering tipe kulit...  Positif   
..                                                 ...      ...   
130                   suka sama warna next order warna   Netral   
131       tipe kulit minyak alhamdulillah cocok thanks   Netral   
132  beli serum pas harga flash sale lumayan banget...  Positif   
133  jujur buat aku enggak cocok aku gapaham skinca...  Negatif   
134         hasil akhir bagus kayak kulit bersih sehat  Negatif   

                                              sentimen  
0    {'classified_text': 'beautiful package bintang...  
1    {'classified_text': 'tipe kulit kering', 'twee...  
2    {'classified_text':

In [64]:
import pandas as pd
import json

# Membaca file CSV
df = pd.read_csv('sentimen_150.csv')

# Nama kolom yang berisi data sentimen
nama_kolom_sentimen = 'sentimen'

# Fungsi untuk memisahkan data JSON dalam kolom sentimen
def parse_sentimen(sentimen_str):
    try:
        sentimen_dict = json.loads(sentimen_str.replace("'", "\""))
        return pd.Series(sentimen_dict)
    except:
        return pd.Series({})

# Memisahkan data dalam kolom 'sentimen' menjadi kolom-kolom terpisah
df_sentimen = df[nama_kolom_sentimen].apply(parse_sentimen)

# Menggabungkan DataFrame baru dengan DataFrame asli
df = pd.concat([df, df_sentimen], axis=1)

# Menyimpan perubahan ke dalam file CSV
df.to_csv('sentimen_150_terpisah.csv', index=False)

# Menampilkan DataFrame setelah perubahan
print(df)


                                 review_tokens_stemmed Sentimen  \
0            beautiful package bintang buat safety nya  Positif   
1                                    tipe kulit kering   Netral   
2    kirim lama beda provinsi gapapa packingnya ama...  Negatif   
3                                                  top  Positif   
4    tipe kulit minyak tipe kulit kering tipe kulit...  Positif   
..                                                 ...      ...   
130                   suka sama warna next order warna   Netral   
131       tipe kulit minyak alhamdulillah cocok thanks   Netral   
132  beli serum pas harga flash sale lumayan banget...  Positif   
133  jujur buat aku enggak cocok aku gapaham skinca...  Negatif   
134         hasil akhir bagus kayak kulit bersih sehat  Negatif   

                                              sentimen  \
0    {'classified_text': 'beautiful package bintang...   
1    {'classified_text': 'tipe kulit kering', 'twee...   
2    {'classified_tex

### CEK JUMLAH REVIEW SETIAP KELAS

In [65]:
# Load data ke dalam data frame 
import pandas as pd

# spesifikasi encoding diperlukan karena data tidak menggunakan UTF-8
df = pd.read_csv('sentimen_150_terpisah.csv', encoding='latin-1') 

df.head()

,review_tokens_stemmed,Sentimen,sentimen,classified_text,tweet_text,sentence_score,max_positive,max_negative,kelas
0,beautiful package bintang buat safety nya,Positif,{'classified_text': 'beautiful package bintang...,beautiful package bintang buat safety nya,beautiful package bintang buat safety nya,['beautiful package bintang buat safety nya'],1,-1,netral
1,tipe kulit kering,Netral,"{'classified_text': 'tipe kulit kering', 'twee...",tipe kulit kering,tipe kulit kering,['tipe kulit kering'],1,-1,netral
2,kirim lama beda provinsi gapapa packingnya ama...,Negatif,{'classified_text': 'kirim lama [-2] beda prov...,kirim lama [-2] beda provinsi gapapa packingny...,kirim lama beda provinsi gapapa packingnya am...,['kirim lama [-2] beda provinsi gapapa packing...,5,-2,positif
3,top,Positif,"{'classified_text': 'top [4]', 'tweet_text': '...",top [4],top,['top [4]'],4,-1,positif
4,tipe kulit minyak tipe kulit kering tipe kulit...,Positif,{'classified_text': 'tipe kulit minyak tipe ku...,tipe kulit minyak tipe kulit kering tipe kulit...,tipe kulit minyak tipe kulit kering tipe kuli...,['tipe kulit minyak tipe kulit kering tipe kul...,3,-5,negatif


In [66]:
# Drop kolom yang tidak terpakai
df = df.drop(df.columns[[2, 3, 4, 5, 6, 7]], axis=1)

# Cek data teratas
df.head()

,review_tokens_stemmed,Sentimen,kelas
0,beautiful package bintang buat safety nya,Positif,netral
1,tipe kulit kering,Netral,netral
2,kirim lama beda provinsi gapapa packingnya ama...,Negatif,positif
3,top,Positif,positif
4,tipe kulit minyak tipe kulit kering tipe kulit...,Positif,negatif


In [67]:
# Data untuk rename kolom v1 dan v2
new_cols = {
    'review_tokens_stemmed' : 'Review',
    'Sentimen' : 'Sentimen-Manual',
    'kelas' : 'Sentimen-SentiStrength'
}

# Rename nama kolom v1 dan v2
df = df.rename(columns=new_cols)

# Cek data teratas
df.head()

,Review,Sentimen-Manual,Sentimen-SentiStrength
0,beautiful package bintang buat safety nya,Positif,netral
1,tipe kulit kering,Netral,netral
2,kirim lama beda provinsi gapapa packingnya ama...,Negatif,positif
3,top,Positif,positif
4,tipe kulit minyak tipe kulit kering tipe kulit...,Positif,negatif


In [68]:
# Cek jumlah data per kelas
print(df['Sentimen-SentiStrength'].value_counts())
print('\n')

# Cek kelengkapan data
print(df.info())
print('\n')

# Cek statistik deskriptif
print(df.describe())

positif    92
netral     29
negatif    14
Name: Sentimen-SentiStrength, dtype: int64


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135 entries, 0 to 134
Data columns (total 3 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Review                  135 non-null    object
 1   Sentimen-Manual         135 non-null    object
 2   Sentimen-SentiStrength  135 non-null    object
dtypes: object(3)
memory usage: 3.3+ KB
None


       Review Sentimen-Manual Sentimen-SentiStrength
count     135             135                    135
unique    130               3                      3
top     bagus          Netral                positif
freq        3              68                     92
